<h1> Problem 66 - Diophantine Equation </h1> 

Consider the quadratic Diophantine equation of the form, 
\begin{equation}
    x^2 - D y^2 = 1
\end{equation}
For example, when $D=13$, the minimal solution in $x$ is $649^2 - 13\times 180^2 = 1$. It can be assumed that there are no solutions in positive integers when $D$ is square. 

By finding the minimal solution in $x$ for $D = (2,3, 4, 6,7)$, the following is obtained, 
\begin{align*}
    3^2 - 2 \times 2^2 &= 1 \\
    2^2 - 3 \times 1^2 &= 1 \\
    9^2 - 5 \times 4^2 &= 1 \\
    5^2 - 6 \times 2^2 &= 1 \\
    8^2 - 7 \times 3^2 &= 1 
\end{align*}
Hence, by considering the minimal solution in $x$ for $D \leq 7$, the largest $x$ is obtained when $D=5$. 

Find the value of $D \leq 1000$ in minimal soution of $x$ for which the largest $x$ is obtained. 

---



There are two methods to solve this. 

The first method is the [Brahmagupta method](https://en.wikipedia.org/wiki/Pell%27s_equation) which utilises the identity, 
\begin{equation}
    (x_1 - Ny_1^2) (x_2^2 - N y_2^2) = (x_1 x_2 + N y_1 y_2)^2 - N (x_1 y_2 + x_2y_1)^2
\end{equation}
Therefore, tripes of the form $(x_1, y_1, k_1)$ and $(x_2, y_2, k_2)$ that are solutions of $x^2 - Ny^2 = k$, to generate a new triple 
\begin{equation}
    (x_1 x_2 + N y_1y_2, x_1y_2 + x_2 y_1, k_1k_2) 
    % \quad \text{and} \quad (x_1 x_2 - N y_1 y_2, x_1 y_2 - x_2 y_1, k_1k_2)
\end{equation}
For any triple $(a,b,k)$ that satisfies $a^2 - Nb^2 = k$, it is composed of the trivial triple $(m, 1, m^2 - N)$ to yield the new triple, 
\begin{equation}
    (am + Nb, a + bm, k(m^2 -N))
\end{equation}
If $a$ and $b$ are relative primes ($\implies gcd(a,b) = 1$), then 
\begin{equation}
    \bigg(\frac{am + Nb}{k}\bigg)^2 - N \bigg(\frac{a + bm}{k}\bigg)^2 = \frac{m^2 - N}{k}
\end{equation}

Therefore, by starting with the trivial solution of $a=m$ and $b=1$, the solution can be achieved by solving it iteratirvely and termination when a new $(a, b, k)$ has $k=1$. Two conditions are imposed on how $m$ is selected, 
1. $\frac{a+bm}{k} = \text{integer}$
2. $m$ minimises $m^2 - N$
Annoyingly, I am unable an efficient method to find $m$. The only method I can think of involves just trying the possible $m$ values. 


The second method involves the use of continued fractions. The solution corresponds to the $k^{th}$ convergents $\big(\frac{p_k}{q_k}\big)$ of $\sqrt{n}$, where 
\begin{equation}
    \sqrt{n} = [q_0; q_1, q_2, \ldots ] = [q_0; \overline{q_1, \ldots, q_m} ] 
\end{equation}
where $q_m = 2q_0$ and the terms from $q_1$ to $q_{m-1}$ is a palindrome. I had originally hoped that there would be a deterministic method to find the length of the priodicity without ever computing the continued fraction. However, I was not able to find one. Therfore, I've concluded that the only method to solve this problem is by manually finding the all the solution for Pell's equation with $d \leq 1000$. 

The algorithm implemented can be found [here](http://www.kurims.kyoto-u.ac.jp/EMIS/journals/GMN/yahoo_site_admin/assets/docs/1_GMN-8492-V28N2.190180001.pdf). [This](https://kconrad.math.uconn.edu/blurbs/ugradnumthy/pelleqn1.pdf), [this](https://kconrad.math.uconn.edu/blurbs/ugradnumthy/pelleqn2.pdf) and [this](http://web.math.princeton.edu/mathlab/jr02fall/Periodicity/alexajp.pdf) are also useful references. 

In [2]:
fn is_square(n: u32) -> bool {
    let sqrt_n_fractional_part: f64 = f64::from(n).sqrt().fract();
    let eps: f64 = 1.0e-7_f64;
    
    if sqrt_n_fractional_part > -eps && sqrt_n_fractional_part < eps {
        true
    } else {
        false
    }
}

In [3]:
fn update_q(d: u32, c_i_minus1: u32, c_i: u32) -> u32 {
    let d_float = f64::from(d);
    let c_i_minus_1_float = f64::from(c_i_minus1);
    let c_i_float = f64::from(c_i);
    
    let q = ((d_float - c_i_minus_1_float*c_i_float).sqrt() + d_float.sqrt())/(c_i_float);
//     println!("{}", c_i_float);
    
    q.floor() as u32
}

In [4]:
fn update_c(d: u32, c_i_minus1: u32, c_i: u32, q_i: u32) -> u32 {
    let d_float = f64::from(d);
    let q_float = f64::from(q_i);
    let c_i_minus_1_float = f64::from(c_i_minus1);
    let c_i_float = f64::from(c_i);
    
    let term_1 = 2.0_f64*q_float*((d_float - c_i_minus_1_float*c_i_float).sqrt());
    let term_3 = q_float*q_float*c_i_float;
    
    (term_1 + c_i_minus_1_float - term_3) as u32
}

In [5]:
#[derive(Clone, Debug)]
struct PellsSolution {
    a: u64, 
    b: u64, 
    c: u32,
    q: Vec<u32>,
}

impl PellsSolution {
    fn new(a: u64, b:u64, c:u32, q: Vec<u32>) -> PellsSolution {
        PellsSolution {a,b,c,q}
    }
    
    fn find_next_a(previous_a: u64, a: u64, q:u32) -> u64 {
        (q as u64)*a + previous_a
    }
    
    fn find_next_b(previous_b: u64, b: u64, q:u32) -> u64 {
        (q as u64)*b + previous_b
    }
    
    fn find_next_c(d: u32, previous_c: u32, c: u32, q:u32) -> u32 {
        update_c(d, previous_c.clone(), c.clone(), q) as u32
    }
    
    fn new_from(prev_iter: &PellsSolution, cur_iter: &PellsSolution, q: u32, d:u32) -> PellsSolution {
//         let new_a = PellsSolution::find_next_a(prev_iter.a.clone(), cur_iter.a.clone(), q);
//         let new_b = PellsSolution::find_next_b(prev_iter.b.clone(), cur_iter.b.clone(), q);
        let new_c = PellsSolution::find_next_c(d, prev_iter.c.clone(), cur_iter.c.clone(), q);
        let mut q_history = cur_iter.q.clone(); 
        q_history.push(q);
//         println!("{} {} {} {}", new_a, new_b, new_c, q);
        
//         PellsSolution::new(new_a, new_b, new_c)
        PellsSolution::new(0, 0, new_c, q_history.clone())
    }
    
    fn replace(replacement: PellsSolution) -> PellsSolution {
        PellsSolution{..replacement}
    }
}

In [6]:
fn solve_pells(d: u32) -> PellsSolution {
//     let mut iter_previous = PellsSolution {a: 0, b: 1, c: d};
    let mut iter_previous = PellsSolution::new(0,1,d, Vec::new());
    let mut iter = PellsSolution::new(1, 0, 1, Vec::new());
    let mut iter_next: PellsSolution;
    let mut q: u32;
    let q0: u32 = f64::from(d).sqrt().floor() as u32;
    let mut m: u32 = 0;
    
    loop {
        q = update_q(d, iter_previous.c.clone(), iter.c.clone());
        iter_next = PellsSolution::new_from(&iter_previous, &iter, q, d);
        
//         if &iter_next.c == &1 {
//             break
//         }
        if q == 2*q0 && (m & 1) == 0{
            return iter
        }
        
        m += 1;
        iter_previous = PellsSolution::replace(iter);
        iter = PellsSolution::replace(iter_next);
    }
    
}

In [7]:
fn largest_fundamental_solution(d_upper_bound: u32) -> u32 {
    let mut max_q_sum: u32 = 0;  
    let mut max_d: u32 = 0;
    
    for i in 2..d_upper_bound + 1 {
        if !is_square(i) {
            let this_sol = solve_pells(i);
            let q_sum = this_sol.q.clone().into_iter().sum();
            if q_sum > max_q_sum {
                max_q_sum = q_sum;
                max_d = i;
//                 println!("{}", i)
            }
        }
    }
    
    max_d
}

In [8]:
let sol = solve_pells(541);
println!("{:?} ", sol);

PellsSolution { a: 0, b: 0, c: 1, q: [23, 3, 1, 5, 1, 8, 2, 4, 1, 2, 3, 1, 1, 11, 15, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 15, 11, 1, 1, 3, 2, 1, 4, 2, 8, 1, 5, 1, 3, 46, 3, 1, 5, 1, 8, 2, 4, 1, 2, 3, 1, 1, 11, 15, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 15, 11, 1, 1, 3, 2, 1, 4, 2, 8, 1, 5, 1, 3] } 


In [9]:
let max_x = largest_fundamental_solution(1000);
println!("For D < 1001, the largest x is {}", max_x);

For D < 1001, the largest x is 661
